In [1]:
import os
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import math

In [2]:
tf.executing_eagerly()
tf.config.list_physical_devices('GPU')

[]

In [3]:
seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)

size_input = 28*28
size_hidden_1 = 256
size_hidden_2 = 128
size_output = 10
batch_size=30
lr=0.2
dropout_p=0.0
L1=0.0000
L2=0.0000

fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test[0:20])
x_train=tf.reshape(x_train,[x_train.shape[0],-1])
x_test=tf.reshape(x_test,[x_test.shape[0],-1])
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test[0:20])

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
[9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0]
(60000, 784)
(60000,)
(10000, 784)
(10000,)
[9 2 1 1 6 1 4 6 5 7 4 5 7 3 4 1 2 4 8 0]


In [4]:
class MLP(tf.keras.Model):
  def __init__(self, size_input, size_hidden_1, size_hidden_2, size_output, device=None):
    super(MLP, self).__init__()
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """

    self.size_input, self.size_hidden_1, self.size_hidden_2, self.size_output, self.device =\
    size_input, size_hidden_1, size_hidden_2, size_output, device

    self.initial=tf.keras.initializers.he_normal(seed=seed)
    
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(self.initial([self.size_input, self.size_hidden_1]))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(self.initial([1, self.size_hidden_1]))
     # Initialize weights between hidden layer and output layer
    self.W2 = tf.Variable(self.initial([self.size_hidden_1, self.size_hidden_2]))
    # Initialize biases for hidden layer
    self.b2 = tf.Variable(self.initial([1, self.size_hidden_2]))
     # Initialize weights between hidden layer and output layer
    self.W3 = tf.Variable(self.initial([self.size_hidden_2, self.size_output]))
    # Initialize biases for output layer
    self.b3 = tf.Variable(self.initial([1, self.size_output]))
    
    # Define variables to be updated during backpropagation
    self.MLP_variables = [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3]

    self.loss_object =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.reg_12=tf.keras.regularizers.L1L2(l1=L1, l2=L2)
    
  def forward(self, training, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if training==1:
      if self.device is not None:
        with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
          self.y = self.compute_output(X)
      else:
        self.y = self.compute_output(X)
    elif training==0:
      if self.device is not None:
        with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
          self.y = self.compute_output_test(X)
      else:
        self.y = self.compute_output_test(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    #y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    return self.loss_object(y_true, y_pred)+self.reg_12(self.W1)+self.reg_12(self.W2)
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    #optimizer = tf.keras.optimizers.Adam()
    with tf.GradientTape() as tape:
      predicted = self.forward(1,X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.MLP_variables)
    optimizer.apply_gradients(zip(grads, self.MLP_variables))

        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    #Remember to normalize your dataset before moving forward
    # Compute values in hidden layer
    what = tf.matmul(X_tf, self.W1) + self.b1
    hhat = tf.nn.relu(what)
    hhat = tf.nn.dropout(hhat, rate = dropout_p, seed = seed)
    what_1 = tf.matmul(hhat, self.W2) + self.b2
    hhat_1 = tf.nn.relu(what_1)
    hhat_1 = tf.nn.dropout(hhat_1, rate = dropout_p, seed = seed)
    # Compute output
    what_2 = tf.matmul(hhat_1, self.W3) + self.b3
    #output= tf.nn.softmax(what_2)
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    return what_2

  def compute_output_test(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    #Remember to normalize your dataset before moving forward
    # Compute values in hidden layer
    what = tf.matmul(X_tf, self.W1) + self.b1
    hhat = tf.nn.relu(what)
    what_1 = tf.matmul(hhat, self.W2) + self.b2
    hhat_1 = tf.nn.relu(what_1)
    # Compute output
    what_2 = tf.matmul(hhat_1, self.W3) + self.b3
    #output= tf.nn.softmax(what_2)
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    return what_2
  

In [5]:
t=np.array([[[1,1],[2,2]],[[3,2],[2,3]]])
print(tf.reshape(t,[2,1,-1]))
t.shape

tf.Tensor(
[[[1 1 2 2]]

 [[3 2 2 3]]], shape=(2, 1, 4), dtype=int32)


(2, 2, 2)

In [6]:
# Set number of epochs
NUM_EPOCHS = 20

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

train_loss.reset_states()
train_accuracy.reset_states()
test_loss.reset_states()
test_accuracy.reset_states()

mlp_on_default = MLP(size_input, size_hidden_1, size_hidden_2, size_output)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000, seed=seed).batch(batch_size)

for inputs, outputs in test_ds:
  preds = mlp_on_default.forward(0,inputs)
  test_loss(mlp_on_default.loss(preds,outputs))
  test_accuracy(outputs, preds)

print(
  f'Epoch {0}, '
  f'Test Loss: {test_loss.result()}, '
  f'Test Accuracy: {test_accuracy.result() * 100}'
)

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  
  for inputs, outputs in train_ds:
    mlp_on_default.backward(inputs, outputs)
    preds = mlp_on_default.forward(0,inputs)
    train_loss(mlp_on_default.loss(preds,outputs))
    train_accuracy(outputs, preds)

  for inputs, outputs in test_ds:
    preds = mlp_on_default.forward(0,inputs)
    test_loss(mlp_on_default.loss(preds,outputs))
    test_accuracy(outputs, preds)
  
  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

Epoch 0, Test Loss: 3.653257131576538, Test Accuracy: 12.050000190734863
Epoch 1, Loss: 0.368476927280426, Accuracy: 87.99333190917969, Test Loss: 0.5058491230010986, Test Accuracy: 81.87999725341797
Epoch 2, Loss: 0.221223846077919, Accuracy: 92.85166931152344, Test Loss: 0.4344072639942169, Test Accuracy: 84.5
Epoch 3, Loss: 0.19239984452724457, Accuracy: 93.89167022705078, Test Loss: 0.3814765512943268, Test Accuracy: 85.98999786376953
Epoch 4, Loss: 0.1747613400220871, Accuracy: 94.52000427246094, Test Loss: 0.37567225098609924, Test Accuracy: 86.05999755859375
Epoch 5, Loss: 0.1641133725643158, Accuracy: 94.85333251953125, Test Loss: 0.37308743596076965, Test Accuracy: 86.52999877929688
Epoch 6, Loss: 0.15412969887256622, Accuracy: 95.29499816894531, Test Loss: 0.37678465247154236, Test Accuracy: 86.5
Epoch 7, Loss: 0.1469857543706894, Accuracy: 95.5616683959961, Test Loss: 0.35576561093330383, Test Accuracy: 86.66999816894531
Epoch 8, Loss: 0.1395873874425888, Accuracy: 95.833328